#### Title:          openpifpaf_annotate_data.ipynb
#### Description:    Merge output files to a single CSV file per video
#### Author:         Fabian Kahl

In [ ]:
from numpy import load
import pandas as pd
import cv2
import os
import json
import numpy as np
from pathlib import Path

In [ ]:
def json_to_csv(json_path, video, csv_dir_out):
    duration = np.nan

    # Source: https://openpifpaf.github.io/datasets.html#coco-person-keypoints
    joint_names = ['nose', 'left_eye', 'right_eye', 'left_ear',
                   'right_ear', 'left_shoulder', 'right_shoulder',
                   'left_elbow', 'right_elbow', 'left_wrist',
                   'right_wrist', 'left_hip', 'right_hip',
                   'left_knee', 'right_knee', 'left_ankle', 'right_ankle']

    dims = ['X', 'Y', 'C']
    cols = []
    for joint in joint_names:
        for suffix in dims:
            cols.append(joint + suffix)

    # Get timestamp and frame
    cap = cv2.VideoCapture(video)
    timestamps = []
    while cap.isOpened():
        success, image = cap.read()
        timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)
        if not success:
            print("Ignoring empty video frame.")
            break
        timestamps.append(timestamp_ms)
    cap.release()

    df = pd.DataFrame()
    with open(json_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            try:
                df_frame_stacked = pd.DataFrame([data['predictions'][0]['keypoints']], columns=cols, index=[data['frame']])
            except:
                print('Not readable: ', video, data)
            # Append df_frame_stacked on df containing all frames
            df = pd.concat([df, df_frame_stacked])

    path_split = video.rsplit('/', 1)
    vid_name_in = path_split[-1]
    csv_name_out = vid_name_in.replace('.mp4', '.csv')

    # Fill some meta data
    vid_in_split = vid_name_in.split('_')
    df['Subject'] = 'Sub'+str(int(vid_in_split[0]))
    df['exerciseName'] = vid_in_split[2]
    df['ipad_view'] = vid_in_split[3].split('.')[0]
    df['duration'] = duration
    df['Timestamp'] = timestamps
    df['Frame'] = list(range(1, len(df)+1))
    # Rearrange the cols to have the meta data in front
    cols = df.columns.to_list()
    df = df[cols[-6:] + cols[:-6]] # Move last 6 cols to the front

    if not os.path.exists(csv_dir_out):
        os.mkdir(csv_dir_out)
    df.to_csv(csv_dir_out + csv_name_out, index=False)

def list_subdirs(folder_path):
    subdirs = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    return subdirs

In [ ]:
results_dir = '../data/DZNE_runs/OpenPifPaf/'
subdirs = list_subdirs(results_dir)
for subdir in subdirs:
    subsubdirs = list_subdirs(results_dir+subdir+'/')
    for subsubdir in subsubdirs:
        json_path = results_dir + subdir + '/' + subsubdir + '/markers.json'
        video = '../data/vid_cut/' + subdir + '/' + subsubdir.replace('.MOV', '.mp4')
        csv_dir_out = '../data/non_area_fileformat/frameworks_orig/OpenPifPaf/' + subdir + '/'
        if not os.path.exists(csv_dir_out):
            os.makedirs(csv_dir_out)
        json_to_csv(json_path, video, csv_dir_out)